# steps

- h3 sort in normal (ascending order)
- this puts hexes in order like `[child_0, child_1, child_2, ..., child_6, parent]`
- then zip through list **backwards** to remove children and duplicates, zero these guys out
- then we use a stack moving up the list **forwards** to find cells we can compact

In [1]:
import numpy as np
import h3

In [2]:
h = h3.geo_to_h3(0,0,5)

In [3]:
h

'85754e67fffffff'

In [4]:
h3.string_to_h3(h)

601042424243945471

In [14]:
def is_first_child(h):
    if h3.h3_get_resolution(h) == 0:
        # res zero cells go right through
        return False
    
    p = h3.h3_to_parent(h)
    children = h3.h3_to_children(p)
    first_child = min(children)
    
    return h == first_child

def rich_compare(a, b):
    if a == b:
        return 0
    
    res_a = h3.h3_get_resolution(a)
    res_b = h3.h3_get_resolution(b)
    
    # need child < parent

    # if b is parent of a
    if res_a > res_b:
        if h3.h3_to_parent(a, res_b) == b:
            return -1

    # if a is parent of b
    if res_a < res_b:
        if h3.h3_to_parent(b, res_a) == a:
            return +1
    
    mask = 0x800f_ffff_ffff_ffff  # not sure why we have the 8 here..?
#     mask = 0x800f_ffff_ffff_ffff
    
    a = h3.string_to_h3(a)
    b = h3.string_to_h3(b)
    if (a & mask) < (b & mask):
        return -2
    else:
        return +2
    
def simple_compare(a, b): 
    a = h3.string_to_h3(a)
    b = h3.string_to_h3(b)
    mask = 0x800f_ffff_ffff_ffff

    # return np.sign((a & mask) - (b & mask))  # also works
    return (a & mask) - (b & mask)


from functools import cmp_to_key
def h3order(hexes):
    hexes = sorted(hexes, key = cmp_to_key(simple_compare))

    return hexes

In [18]:
# hexes =  list(h3.h3_to_children(a, 2)) + list(h3.h3_to_children(a, 1))

b = list(h3.get_res0_indexes())[0]

hexes =  [b] + list(h3.h3_to_children(b, 1)) + list(h3.h3_to_children(b, 2))

sorted(hexes, key = cmp_to_key(simple_compare)) == sorted(hexes, key = cmp_to_key(rich_compare))

True

In [19]:
# def filter_dupes_and_children(seq):
#     """ Assume sequence in descending h3-order.
#     """
#     prev = None
#     prev_res = None
    
#     for e in seq:
#         if prev is None:
#             prev = e
#             prev_res = h3.h3_get_resolution(e)
#             yield e
#         else:
#             cur_res = h3.h3_get_resolution(e)
            
#             if cur_res >= prev_res and h3.h3_to_parent(e, prev_res):
#                 # then e is child of prev
#                 continue
#             else:
#                 prev = e
#                 prev_res = cur_res
#                 yield e
                

# h3 descending order

When we put hexes in h3 order (descending) we get an order like

[parent, child_6, child_5, child_4, ..., child_0]

In [107]:
h3order_descending([a] + list(h3.h3_to_children(a, 1)))

['8065fffffffffff',
 '8165bffffffffff',
 '81657ffffffffff',
 '81653ffffffffff',
 '8164fffffffffff',
 '8164bffffffffff',
 '81647ffffffffff',
 '81643ffffffffff']

In [102]:
out = sorted(hexes, key = cmp_to_key(rich_compare), reverse=True)
#list(filter_dupes_and_children(out))
out

['8065fffffffffff',
 '8165bffffffffff',
 '8265b7fffffffff',
 '8265affffffffff',
 '8265a7fffffffff',
 '82659ffffffffff',
 '826597fffffffff',
 '82658ffffffffff',
 '826587fffffffff',
 '81657ffffffffff',
 '826577fffffffff',
 '82656ffffffffff',
 '826567fffffffff',
 '82655ffffffffff',
 '826557fffffffff',
 '82654ffffffffff',
 '826547fffffffff',
 '81653ffffffffff',
 '826537fffffffff',
 '82652ffffffffff',
 '826527fffffffff',
 '82651ffffffffff',
 '826517fffffffff',
 '82650ffffffffff',
 '826507fffffffff',
 '8164fffffffffff',
 '8264f7fffffffff',
 '8264effffffffff',
 '8264e7fffffffff',
 '8264dffffffffff',
 '8264d7fffffffff',
 '8264cffffffffff',
 '8264c7fffffffff',
 '8164bffffffffff',
 '8264b7fffffffff',
 '8264affffffffff',
 '8264a7fffffffff',
 '82649ffffffffff',
 '826497fffffffff',
 '82648ffffffffff',
 '826487fffffffff',
 '81647ffffffffff',
 '826477fffffffff',
 '82646ffffffffff',
 '826467fffffffff',
 '82645ffffffffff',
 '826457fffffffff',
 '82644ffffffffff',
 '826447fffffffff',
 '81643ffffffffff',


In [79]:
i = 0

rich_compare(out[i], out[i+1])

-2

In [80]:
p = '8065fffffffffff'
c = '81643ffffffffff'

assert p == h3.h3_to_parent(c)

In [81]:
children = sorted(h3.h3_to_children(p))

for c in children:
    assert simple_compare(c,p) < 0 # child less than parent

In [82]:
simple_compare(c,p)

-1

In [83]:
rich_compare(c,p)

-1